In [2]:
import sqlite3
import pandas as pd


In [3]:
db_path = "Data Engineer_ETL Assignment.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [11]:
query = """
select c.customer_id,c.age,i.item_name,CAST(COALESCE(SUM(o.quantity), 0) AS INTEGER) AS Quantity 
from orders o left join items i on o.item_id = i.item_id
left join sales s on o.sales_id = s.sales_id
left join customers c on s.customer_id = c.customer_id
where c.age >=18
and c.age <=35
group by 1,2,3
having CAST(COALESCE(SUM(o.quantity), 0) AS INTEGER) > 0

"""

In [12]:
df_results = pd.read_sql_query(query,conn)
df_results

,customer_id,age,item_name,Quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
65,97,23,y,2
66,97,23,z,3
67,100,30,x,1
68,100,30,y,1


In [6]:
df_sales = pd.read_sql_query("SELECT * FROM sales", conn)
df_customers = pd.read_sql_query("SELECT * FROM customers", conn)
df_orders = pd.read_sql_query("SELECT * FROM orders", conn)
df_items = pd.read_sql_query("SELECT * FROM items", conn)


In [17]:
df = (
    df_orders
    .merge(df_items, on="item_id", how="left")
    .merge(df_sales, on="sales_id", how="left")
    .merge(df_customers, on="customer_id", how="left")
)

df = df[(df["age"] >= 18) & (df["age"] <= 35)]

result = (
    df.groupby(["customer_id", "age", "item_name"], as_index=False)
      .agg(Quantity=("quantity", "sum"))
)

result["Quantity"] = result["Quantity"].fillna(0).astype(int)

result = result[result["Quantity"]>0]

In [18]:
result

,customer_id,age,item_name,Quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
67,97,23,y,2
68,97,23,z,3
69,100,30,x,1
70,100,30,y,1
